# Notebook for CE Anlaysis using markov chain models
#### Brandon Chan July 2020

### Import packages

In [1]:
import sys
sys.path.insert(0,"../src")

import pandas as pd # Dataframe structure and reading of excel files
import numpy as np # Scientific computing - used for n-dimensional arrays, numerical functions
import matplotlib.pyplot as plt # Plotting package
import seaborn as sns # Another plotting package
import os # for file path things
from arch.bootstrap import IIDBootstrap # Bootstrap analysis
from markov_modeling import run_model # code for running/iterating model

### Set cycle length (needed to adjust to QALY)
TODO:// read in specification file(s) to set this variable

In [2]:
#---------------------------------------------------------------------------------------------------
# Read in excel workbook and assign each sheet to a dataframe
#---------------------------------------------------------------------------------------------------
specification_df = pd.read_excel('../model_specifications/test_parameters_base.xlsx', 'specification', header=None, index_col=0)
cycle_length = specification_df.loc['cycle_length'].values[0]

### Load existing model outputs if they exist

In [5]:
pop_base = np.load('../model_outputs/test_base_20-07-2020_population.npy')
cost_base = np.load('../model_outputs/test_base_20-07-2020_costs.npy')
util_base = np.load('../model_outputs/test_base_20-07-2020_utilities.npy')

pop_treat = np.load('../model_outputs/test_treat_20-07-2020_population.npy')
cost_treat = np.load('../model_outputs/test_treat_20-07-2020_costs.npy')
util_treat = np.load('../model_outputs/test_treat_20-07-2020_utilities.npy')

### Run model for base-case and treatment arms based on structure and paramters outlined in excel files

In [3]:
pop_base, cost_base, util_base = run_model('../model_specifications/test_parameters_base.xlsx', save=False, model_name='test_base')
pop_treat, cost_treat, util_treat = run_model('../model_specifications/../model_specifications/test_parameters_treat.xlsx', save=False, model_name='test_treat')

file and parameters loaded...
beginning iterations...
model done...
total time: 0.3 seconds || mean time per iteration: 0.0 seconds
calculating costs and utilities...
done costs and utilities...

file and parameters loaded...
beginning iterations...
model done...
total time: 0.31 seconds || mean time per iteration: 0.0 seconds
calculating costs and utilities...
done costs and utilities...



### Consense raw model outputs into desired numbers

In [4]:
#calculate sum at each time point (assign to bottom row)
# result array shape: [num_iterations, num_timesteps]
# each row is the sum of the costs/utilitys at time point t of an iteration
cost_sum_per_cycle_base = np.sum(cost_base, axis=1)
utility_sum_per_cycle_base = np.sum(util_base, axis=1)

cost_sum_per_cycle_treat = np.sum(cost_treat, axis=1)
utility_sum_per_cycle_treat = np.sum(util_treat, axis=1)

# Adjusting utility to QALY 
# = cycle_length /  days_in_a_year (where cycle_length is in days)
utility_sum_per_cycle_base = utility_sum_per_cycle_base * (cycle_length/365)
utility_sum_per_cycle_treat = utility_sum_per_cycle_treat * (cycle_length/365)

# Calculate the total cost of an iteration
cost_sum_per_iteration_base = np.sum(cost_sum_per_cycle_base, axis=1)
utility_sum_per_iteration_base = np.sum(utility_sum_per_cycle_base, axis=1)

cost_sum_per_iteration_treat = np.sum(cost_sum_per_cycle_treat, axis=1)
utility_sum_per_iteration_treat = np.sum(utility_sum_per_cycle_treat, axis=1)

# Difference in costs/utilities when comparing each arm per iteration
c = cost_sum_per_iteration_treat - cost_sum_per_iteration_base
u = utility_sum_per_iteration_treat - utility_sum_per_iteration_base

In [5]:
# Make the CE plane plot
plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.ylabel(r'$\Delta$C', rotation=0, fontsize=14, labelpad=14)
plt.xlabel(r'$\Delta$E', fontsize=14)
sns.scatterplot(x=u, y=c)

# Plots some threshold line...
x = np.linspace(min(u), max(u), 100)
y = 20000*x
plt.plot(x, y, '-r')

'''
cu = max(c)
cl = min(c)
uu = max(u)
ul = min(u)
d = [(uu,uu),(cl,cu),'b',
     (uu,ul),(cl,cl),'b',
     (ul,uu),(cu,cu),'b',
     (ul,ul),(cl,cu),'b']
plt.plot(*d)

mx = utility_sum_per_iteration_treat.mean() - utility_sum_per_iteration_base.mean() #Average util of treat - average util of basecase
my = cost_sum_per_iteration_treat.mean() - cost_sum_per_iteration_base.mean() #Average cost of treat - average cost of basecase
plt.plot(mx, my, 'r*')
'''

plt.ylim(min(c)-50, max(c)+50) # Add padding to axis limits

plt.show()

### ICER

In [6]:
delta_mean_utility = utility_sum_per_iteration_treat.mean() - utility_sum_per_iteration_base.mean() #Average util of treat - average util of basecase
delta_mean_cost = cost_sum_per_iteration_treat.mean() - cost_sum_per_iteration_base.mean() #Average cost of treat - average cost of basecase

delta_mean_cost/delta_mean_utility

-488.80280270267417

### Bootstrap for 95% CI

In [7]:
def func(x):
    '''
    Function to return the ICER of the average of the boot strap sample.
    Uses globally defined cost and utility arrays (c and u):
        c and u are [1 x number_of_iterations] numpy arrays that represent 
        the sum of a iterations costs and utility respectivley
    x = indicies of interations to use from bootstrap function
    '''
    return c[x].mean()/u[x].mean()

bs = IIDBootstrap(np.array(list(range(0,c.shape[0])))) #use a "dummy" of array indicies to sample from. Needed to correctly calculate ICER of the average
ci = bs.conf_int(func, 1000, method='bca') #bias-corrected and accelerated method

#print(ci)

print(round(delta_mean_cost/delta_mean_utility,2),'(',round(ci[0][0],2),'to',round(ci[1][0],2),')')

-488.8 ( -507.62 to -470.83 )


### Example using just the cea_analysis() function from analysis_helpers.py

In [1]:
import sys
sys.path.insert(0,"../src")

from analysis_helpers import cea_analysis

In [2]:
cea_analysis('../model_specifications/test_parameters_base.xlsx','../model_specifications/test_parameters_test.xlsx')

file and parameters loaded...
beginning iterations...
model done...
total time: 0.31 seconds || mean time per iteration: 0.0 seconds
calculating costs and utilities...
done costs and utilities...

file and parameters loaded...
beginning iterations...
model done...
total time: 0.31 seconds || mean time per iteration: 0.0 seconds
calculating costs and utilities...
done costs and utilities...

-488.01 ( -507.15 to -469.85 )
